In [1]:
import numpy as np

In [2]:
from PIL import Image

In [9]:
im1 = Image.open(r'/home/aman/Desktop/software-engg-lab-2022S/PythonDS-Assignment/data/imgs/3.jpg')

In [10]:
im1.show()

Segmentation fault (core dumped)
